<a href="https://colab.research.google.com/github/tyukei/AI/blob/master/report2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#第2回 レポート課題 (12/20 23:55 締切)


# 課題1
### トピック9のnotebook1で示したRNNを用いたIMDBデータのP/N判定のコードを参考に、複数のネットワーク構造を設定して性能の違いを調べよ。また、調べたネットワーク構造の中から比較的性能が高くて学習時間が短いものをひとつ選び、学習データが極端に少ない状況からすべての学習データを用いる状況まで、段階的にデータを増やして学習データの量と識別器の性能の関係を調べよ。 

## 準備

ディープニューラルネットワークでの識別を Keras でコーディングします。Google ColabでGPUを使用するときは、「ランタイム」->「ランタイムのタイプを変更」-> ハードウェアアクセラレータ -> GPU を選ぶ。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime, os
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import np_utils
from tensorflow.keras import layers

In [2]:
%load_ext tensorboard

## データセットについて

IMDBデータは映画のレビューに対して、P/N(肯定/否定)のラベルが付いた学習データである。

ここでは、頻度上位10000語を対象とし、データの大きさは先頭の50単語に限定する。

In [3]:
max_features = 10000
maxlen = 50
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)
X_train

17464789/17464789 [==============================] - 2s 0us/step


array([[2071,   56,   26, ...,   19,  178,   32],
       [8255,    5,    2, ...,   16,  145,   95],
       [ 215,   28,  610, ...,    7,  129,  113],
       ...,
       [   4,   65,  496, ...,    4, 3586,    2],
       [  13,   18,   31, ...,   12,    9,   23],
       [7585,    8, 2197, ...,  204,  131,    9]], dtype=int32)

データは単語ではなく、数字が入っている。これは、次のword_indexのindexに相当し、単語はword_indexのvalueに相当する。

In [4]:
word_index = keras.datasets.imdb.get_word_index()

1641221/1641221 [==============================] - 0s 0us/step


単語インデックスを単語に戻して、元のデータにデコードする

In [5]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_train[0]])
decoded_review

"grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all"

```
grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all
```
は「子供は素晴らしく、絶賛されるべきだ」というポジティブなレビューであった。

出力は0か1である。0がネガティブで1がポジティブな映画のレビューとなっている。

In [6]:
y_train[0]

1

出力は１でポジティブとなっている。

確かにレビューとネガポジ判定は一致していて、データセットはうまく機能していると分かる。


## SimpleRNNを構成して学習させる。

RNNとはRecursive Neural Network(再帰型ニューラルネットワーク)の略である。

Recursiveとあるように中間層で演算結果を再帰的に自身に出力する

具体的には、一時刻前の中間層の出力を次の入力と結合して中間層に入力する。

そのため、前の単語が次の単語に影響をあたえる自然言語処理に適していると言われる。

RNNの学習方法は３ステップである

1. ネットワークの構造の決定
1. fix関数を用いて学習実行
1. テストデータを用いて評価

### ネットワークの構造

```
layers.Embedding(max_features, 128)
```
入力の単語インデックスをone-hotベクトル（インデックスに対応する次元の値が1、それ以外の次元の値は0）とみなしたものを低次元のベクトルに変換するEmbedding（埋め込み）層
```
layers.SimpleRNN(64)
```
 Embeddin層の出力とひとつ前の自身の出力を結合したものに対して重み付き和を求めて活性化関数を適用するSimpleRNN層
 ```
layers.Dense(1, activation='sigmoid')
 ```
 SimpleRNN層の出力の重み付き和を求めて活性化関数を適用し、出力を求めるDense層からなる。KerasではSequentialクラスのインスタンスに対してaddメソッドでこれらの層を積み重ねて識別のためのニューラルネットを構成する。

In [7]:
model1 = keras.Sequential([
    layers.Embedding(max_features, 128),#埋め込み層
    layers.SimpleRNN(64),#outputの次元
    layers.Dense(1, activation='sigmoid')#全結合層
]) 
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                12352     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1,292,417
Trainable params: 1,292,417
Non-trainable params: 0
_________________________________________________________________


### 学習
作成したネットワークのcompileメソッドで最適化器・損失関数・評価基準を指定した後、fitメソッドに学習データと学習時のパラメータを与えて学習する。

学習を行う。fit関数を用いる。
主な引数は以下である。 
```
fit(X_train, y_train, epochs=5, batch_size=32,validation_split=0.2)
X_train => 訓練データ
y_train => 教師ラベル
epochs => エポック数
batch_size => バッチサイズ
validation_split => X_trainをさらに分割し、訓練データと検証用に分ける割合
```

fitメソッドの引数validation_splitを指定すると、指定された割合を検証用データとして学習データから分割できる。

1epoch毎に検証用データの判定性能を表示させることができる。

ちなみに、エポック数とは学習回数を表す。
エポック数が少ないと、パラメータが適切に収束する前に学習が終了する。また、あまりにも学習をしすぎると特定のデータのみに強い「過学習」を起こしてしまう。また、学習にも時間がかかる。

バッチサイズは学習データを分割してグループ化(バッチ)したときのデータ数を示す。
バッチサイズが大きいと、局所的解を避けることができ、学習時間を短くすることができる一方で、入力データが平均化され、データの個々の特徴が失われる。一方サイズが小さいと、個々の特徴が反映されるが、時間がかかり、局所解のトラップにかかる可能性が高い。


In [8]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])#訓練プロセス(最適化方、損失関数)
model1.fit(X_train, y_train, epochs=5, batch_size=32,validation_split=0.2)#訓練開始(epochs でエポック,バッチサイズを決める)

Epoch 1/5
625/625 [==============================] - 25s 36ms/step - loss: 0.5554 - acc: 0.6949 - val_loss: 0.4695 - val_acc: 0.7738
Epoch 2/5
625/625 [==============================] - 22s 35ms/step - loss: 0.3272 - acc: 0.8623 - val_loss: 0.5247 - val_acc: 0.7454
Epoch 3/5
625/625 [==============================] - 23s 37ms/step - loss: 0.1482 - acc: 0.9463 - val_loss: 0.6566 - val_acc: 0.7416
Epoch 4/5
625/625 [==============================] - 22s 34ms/step - loss: 0.0564 - acc: 0.9814 - val_loss: 0.8510 - val_acc: 0.7654
Epoch 5/5
625/625 [==============================] - 22s 35ms/step - loss: 0.0257 - acc: 0.9920 - val_loss: 0.9386 - val_acc: 0.7518


### 評価
テストデータを用いて評価を行う。
１に近いほど正答率が高い

In [9]:
test_loss, test_acc = model1.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

782/782 [==============================] - 5s 6ms/step - loss: 0.9539 - acc: 0.7501
Test accuracy: 0.7501


## 双方向LSTM
LSTMはLong Short Term Memory(長・短期記憶)の略である

RNNの勾配消失問題を解消するために考えられたネットワークシステムである。
RNNはデータ数が多いと勾配消失し重みを更新ができなくなる。

RNNが情報をそのまま渡すがLSTMでは中間層を用いて次に渡す。

SimpleRNNでは```layers.SimpleRNN(64)```としていたとろを``` layers.Bidirectional(layers.LSTM(64))```に変更する。引数64はoutputの層の数を示す。

SimpleRNNと同様に、ネットワークの構造決定、学習、評価を行う

In [10]:
model2 = keras.Sequential([
    layers.Embedding(max_features, 128),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [13]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model2.fit(X_train, y_train, epochs=5, batch_size=32,validation_split=0.2)

Epoch 1/5
625/625 [==============================] - 8s 10ms/step - loss: 0.2017 - acc: 0.9224 - val_loss: 0.5042 - val_acc: 0.7916
Epoch 2/5
625/625 [==============================] - 6s 9ms/step - loss: 0.1105 - acc: 0.9611 - val_loss: 0.6703 - val_acc: 0.7852
Epoch 3/5
625/625 [==============================] - 6s 9ms/step - loss: 0.0561 - acc: 0.9814 - val_loss: 0.8881 - val_acc: 0.7776
Epoch 4/5
625/625 [==============================] - 6s 9ms/step - loss: 0.0276 - acc: 0.9918 - val_loss: 1.0400 - val_acc: 0.7694
Epoch 5/5
625/625 [==============================] - 6s 9ms/step - loss: 0.0237 - acc: 0.9923 - val_loss: 1.0861 - val_acc: 0.7730


In [14]:
test_loss, test_acc = model2.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

782/782 [==============================] - 3s 4ms/step - loss: 1.0576 - acc: 0.7821
Test accuracy: 0.7821


# 課題2
### トピック9のnotebook2のコードを参考に、事前学習モデルを用いる設定で、学習データが極端に少ない状況からすべての学習データを用いる状況まで、段階的にデータを増やして学習データの量と識別器の性能の関係を調べよ。